# Train SAR on MovieLens with Azure Machine Learning (Python, CPU)
---
## Introduction to Azure Machine Learning  
The **[Azure Machine Learning service (AzureML)](https://docs.microsoft.com/azure/machine-learning/service/overview-what-is-azure-ml)** provides a cloud-based environment you can use to prep data, train, test, deploy, manage, and track machine learning models. By using Azure Machine Learning service, you can start training on your local machine and then scale out to the cloud. With many available compute targets, like [Azure Machine Learning Compute](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) and [Azure Databricks](https://docs.microsoft.com/en-us/azure/azure-databricks/what-is-azure-databricks), and with [advanced hyperparameter tuning services](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters), you can build better models faster by using the power of the cloud.

Data scientists and AI developers use the main [Azure Machine Learning Python SDK](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py) to build and run machine learning workflows with the Azure Machine Learning service. You can interact with the service in any Python environment, including Jupyter Notebooks or your favorite Python IDE. The Azure Machine Learning SDK allows you the choice of using local or cloud compute resources, while managing and maintaining the complete data science workflow from the cloud.
![AzureML Workflow](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/overview-what-is-azure-ml/aml.png)

This notebook provides an example of how to utilize and evaluate the Simple Algorithm for Recommendation (SAR) algorithm using the Azure Machine Learning service. It takes the content of the [SAR quickstart notebook](sar_movielens.ipynb) and demonstrates how to use the power of the cloud to manage data, switch to powerful GPU machines, and monitor runs while training a model. 

See the hyperparameter tuning notebook for more advanced use cases with AzureML.

### Advantages of using AzureML:
- Manage cloud resources for monitoring, logging, and organizing your machine learning experiments.
- Train models either locally or by using cloud resources, including GPU-accelerated model training.
- Easy to scale out when dataset grows - by just creating and pointing to new compute target

---
## Details of SAR
<details>
    <summary>Click to expand</summary>
    
SAR is a fast scalable adaptive algorithm for personalized recommendations based on user transaction history. It produces easily explainable / interpretable recommendations and handles "cold item" and "semi-cold user" scenarios. SAR is a kind of neighborhood based algorithm (as discussed in [Recommender Systems by Aggarwal](https://dl.acm.org/citation.cfm?id=2931100)) which is intended for ranking top items for each user. 

SAR recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users who have interacted with one item are also likely to have interacted with another. A user has an ***affinity*** to an item if they have interacted with it in the past.

### Advantages of SAR:
- High accuracy for an easy to train and deploy algorithm
- Fast training, only requiring simple counting to construct matrices used at prediction time
- Fast scoring, only involving multiplication of the similarity matric with an affinity vector

### Notes to use SAR properly:
- SAR does not use item or user features, so cannot handle cold-start use cases
- SAR requires the creation of an $mxm$ dense matrix (where $m$ is the number of items). So memory consumption can be an issue with large numbers of items.
- SAR is best used for ranking items per user, as the scale of predicted ratings may be different from the input range and will differ across users.
For more details see the deep dive notebook on SAR here: [SAR Deep Dive Notebook](../02_model_collaborative_filtering/sar_deep_dive.ipynb)</details>
---
## Prerequisities
   - **Azure Subscription**
     - If you don’t have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning service today](https://azure.microsoft.com/en-us/free/services/machine-learning/).
     - You get credits to spend on Azure services, which will easily cover the cost of running this example notebook. After they're used up, you can keep the account and use [free Azure services](https://azure.microsoft.com/en-us/free/). Your credit card is never charged unless you explicitly change your settings and ask to be charged. Or [activate MSDN subscriber benefits](https://azure.microsoft.com/en-us/pricing/member-offers/credit-for-visual-studio-subscribers/), which give you credits every month that you can use for paid Azure services.
---   

In [1]:
# set the environment path to find Recommenders
import os
import shutil
import numpy as np
from tempfile import TemporaryDirectory
from recommenders.datasets import movielens

ModuleNotFoundError: No module named 'recommenders'

In [ ]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### Connect to an AzureML workspace

An [AzureML Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows. In particular, an Azure ML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, deployment, inferencing, and the monitoring of deployed models.

The function below will get or create an AzureML Workspace and save the configuration to `aml_config/config.json`.

It defaults to use provided input parameters or environment variables for the Workspace configuration values. Otherwise, it will use an existing configuration file (either at `./aml_config/config.json` or a path specified by the config_path parameter).

Lastly, if the workspace does not exist, one will be created for you. See [this tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace#portal) to locate information such as subscription id.

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace, Data, Datastore, Environment
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes
import mlflow

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

azureml_mlflow_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri

mlflow.set_tracking_uri(azureml_mlflow_uri)

datastore = ml_client.datastores.get(name='workspaceblobstore')

### Set experiment name
All MLflow runs are logged to the active experiment. By default, runs are logged to an experiment named Default that is automatically created for you. To configure the experiment you want to work on use MLflow command mlflow.set_experiment().

In [ ]:
experiment_name = 'SAR-movielens-job'
mlflow.set_experiment(experiment_name)

### Create a Temporary Directory
This directory will house the data and scripts needed by the AzureML Workspace

In [ ]:
tmp_dir = TemporaryDirectory()

### Download dataset and upload to datastore

Every workspace comes with a default [datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data) (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and access it from the compute target.

The data files are uploaded into a directory named `data` at the root of the datastore.

In [ ]:
TARGET_DIR = 'movielens'

# download dataset
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=['UserId','MovieId','Rating','Timestamp']
)

# upload dataset to workspace datastore
data_file_name = "movielens_" + MOVIELENS_DATA_SIZE + "_data.csv"
# data.to_pickle(os.path.join(tmp_dir.name, data_file_name))
data.to_csv(os.path.join(tmp_dir.name, data_file_name))


In [ ]:
import csv

ds = datastore
my_path = os.path.join(tmp_dir.name, data_file_name)

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description= "Movielens size " + MOVIELENS_DATA_SIZE,
    name="movielens-"+ MOVIELENS_DATA_SIZE + "-dataset",
    # version='0.0.2',
    tags={
        "Source":"Recommenders",
        "Format":"CSV",
    }
)

ml_client.data.create_or_update(my_data)


### Create or Attach Azure Machine Learning Compute 

We create a cpu cluster as our **remote compute target**. If a cluster with the same name already exists in your workspace, the script will load it instead. You can read [Set up compute targets for model training](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets) to learn more about setting up compute target on different locations. You can also create GPU machines when larger machines are necessary to train the model.

According to Azure [Pricing calculator](https://azure.microsoft.com/en-us/pricing/calculator/), with example VM size `STANDARD_D2_V2`, it costs a few dollars to run this notebook, which is well covered by Azure new subscription credit. For billing and pricing questions, please contact [Azure support](https://azure.microsoft.com/en-us/support/options/).

**Note**:
- 10m and 20m dataset requires more capacity than `STANDARD_D2_V2`, such as `STANDARD_NC6` or `STANDARD_NC12`. See list of all available VM sizes [here](https://docs.microsoft.com/en-us/azure/templates/Microsoft.Compute/2018-10-01/virtualMachines?toc=%2Fen-us%2Fazure%2Fazure-resource-manager%2Ftoc.json&bc=%2Fen-us%2Fazure%2Fbread%2Ftoc.json#hardwareprofile-object).
- As with other Azure services, there are limits on certain resources (e.g. AzureML Compute quota) associated with the Azure Machine Learning service. Please read [these instructions](https://docs.microsoft.com/en-us/azure/azure-supportability/resource-manager-core-quotas-request) on the default limits and how to request more quota.
---
#### Learn more about Azure Machine Learning Compute
<details>
    <summary>Click to learn more about compute types</summary>
    
[Azure Machine Learning Compute](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created within your workspace region and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user.

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service.

You can provision a persistent AzureML Compute resource by simply defining two parameters thanks to smart defaults. By default it autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

In addition to vm_size and max_nodes, you can specify:
- **min_nodes**: Minimum nodes (default 0 nodes) to downscale to while running a job on AzureML Compute
- **vm_priority**: Choose between 'dedicated' (default) and 'lowpriority' VMs when provisioning AzureML Compute. Low Priority VMs use Azure's excess capacity and are thus cheaper but risk your run being pre-empted
- **idle_seconds_before_scaledown**: Idle time (default 120 seconds) to wait after run completion before auto-scaling to min_nodes
- **vnet_resourcegroup_name**: Resource group of the existing VNet within which Azure MLCompute should be provisioned
- **vnet_name**: Name of VNet
- **subnet_name**: Name of SubNet within the VNet
</details>
---

In [ ]:
from azure.ai.ml.entities import AmlCompute
cpu_cluster_name = "cpu-cluster"
cluster_basic = AmlCompute(
    name=cpu_cluster_name,
    type="amlcompute",
    size="STANDARD_DS3_v2",
    max_instances=4
)
ml_client.begin_create_or_update(cluster_basic)

# Prepare training script
### 1. Create a directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [ ]:
SCRIPT_DIR = os.path.join(tmp_dir.name, 'movielens-sar')
os.makedirs(SCRIPT_DIR, exist_ok=True)
TRAIN_FILE = os.path.join(SCRIPT_DIR, 'train.py')

### 2. Create a training script
To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train.py` in temporary directory. This training adds a regularization rate to the training algorithm, so produces a slightly different model than the local version.

This code takes what is in the local quickstart and convert it to one single training script. We use run.log() to record parameters to the run. We will be able to review and compare these measures in the Azure Portal at a later time.

In [ ]:
%%writefile $TRAIN_FILE

import argparse
import os
import numpy as np
import pandas as pd
import itertools
import logging

# from azureml.core import Run
import mlflow
import joblib

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.sar import SAR


logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')


TARGET_DIR = 'movielens'
OUTPUT_FILE_NAME = 'outputs/movielens_sar_model.pkl'
MODEL_FILE_NAME = 'movielens_sar_model.pkl'


# get hold of the current run
# run = Run.get_context()
mlflow.autolog()

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data_folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--data_file', type=str, dest='data_file', help='data file name')
parser.add_argument('--top_k', type=int, dest='top_k', default=10, help='top k items to recommend')
parser.add_argument('--data_size', type=str, dest='data_size', default=10, help='Movielens data size: 100k, 1m, 10m, or 20m')
args = parser.parse_args()

# set col names
header = {
    "col_user": "UserId",
    "col_item": "MovieId",
    "col_rating": "Rating",
    "col_timestamp": "Timestamp",
}

# read data
data_pickle_path = os.path.join(args.data_folder, args.data_file)
data = pd.read_pickle(path=data_pickle_path)

# # Log arguments to the run for tracking
# mlflow.log_metric("top-k", args.top_k)
# mlflow.log_metric("data-size", args.data_size)
mlflow.log_param("top-k", args.top_k)
mlflow.log_param("data-size", args.data_size)

# split dataset into train and test
train, test = python_stratified_split(data, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42)

# instantiate the model
model = SAR(
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

# train the SAR model
with Timer() as t:
    model.fit(train)

mlflow.log_metric(name="Training time", value=t.interval)

# predict top k items
with Timer() as t:
    top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

mlflow.log_metric(name="Prediction time", value=t.interval)

# compute evaluation metrics
eval_map = map_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                    col_rating="Rating", col_prediction="prediction", 
                    relevancy_method="top_k", k=args.top_k)
eval_ndcg = ndcg_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                      col_rating="Rating", col_prediction="prediction", 
                      relevancy_method="top_k", k=args.top_k)
eval_precision = precision_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                                col_rating="Rating", col_prediction="prediction", 
                                relevancy_method="top_k", k=args.top_k)
eval_recall = recall_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                          col_rating="Rating", col_prediction="prediction", 
                          relevancy_method="top_k", k=args.top_k)

mlflow.log_metric("map", eval_map)
mlflow.log_metric("ndcg", eval_ndcg)
mlflow.log_metric("precision", eval_precision)
mlflow.log_metric("recall", eval_recall)

# automatic upload of everything in ./output folder doesn't work for very large model file
# model file has to be saved to a temp location, then uploaded by upload_file function
joblib.dump(value=model, filename=MODEL_FILE_NAME)

mlflow.log_artifact(OUTPUT_FILE_NAME, MODEL_FILE_NAME)

In [ ]:
# copy dependent python files
UTILS_DIR = os.path.join(SCRIPT_DIR, 'recommenders')
if os.path.exists(UTILS_DIR):
    shutil.rmtree(UTILS_DIR)
shutil.copytree('../recommenders/', UTILS_DIR)

# Run training script
In this tutorial, this target is AzureML Compute Cluster. All files in the script folder are uploaded into the cluster nodes for execution. `ds.as_mount()` mounts a datastore on the remote compute and returns the folder. See documentation [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data#access-datastores-during-training).

In [ ]:
# script_params = {
#     '--data-folder': ds.container_name,
#     '--data-file': 'movielens/' + data_file_name,
#     '--top-k': TOP_K,
#     '--data-size': MOVIELENS_DATA_SIZE
# }

In [ ]:

ENV_FILE = os.path.join(SCRIPT_DIR, 'environment.yml')

In [ ]:
env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="../environment.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

In [ ]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data, Environment
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import MLClient


# Possible Asset Types for Data:
# AssetTypes.URI_FILE
# AssetTypes.URI_FOLDER
# AssetTypes.MLTABLE

# Possible Paths for Data:
# Blob: https://<account_name>.blob.core.windows.net/<container_name>/<folder>/<file>
# Datastore: azureml://datastores/paths/<folder>/<file>
# Data Asset: azureml:<my_data>:<version>

my_job_inputs = {
    'data_folder': Input(type=AssetTypes.URI_FILE, path=my_path),
    'data_file': 'movielens/' + data_file_name,
    'top_k': TOP_K,
    'data_size': MOVIELENS_DATA_SIZE
}

job = command(
    code=SCRIPT_DIR,  # local path where the code is stored
    # command="python ${{inputs.data_folder}} ${{inputs.data_file}} ${{inputs.top_k}} ${{inputs.data_size}} train.py",
    command="python train.py ${{inputs.data_folder}} ${{inputs.data_file}}",
    inputs=my_job_inputs,
    environment="docker-image-plus-conda-example:6",
    compute="cpu-cluster",
    experiment_name=experiment_name,
)

# submit the command
returned_job = ml_client.jobs.create_or_update(job)
# get a URL for the status of the job
returned_job.services["Studio"].endpoint

## Deploy endpoints using v2
There are several deployment funnels such as managed online endpoints, kubernetes online endpoints (including Azure Kubernetes Services and Arc-enabled Kubernetes) in v2, and Azure Container Instances (ACI) and Kubernetes Services (AKS) webservices in v1. In this article, we'll focus on the comparison of deploying to ACI webservices (v1) and managed online endpoints (v2).